# Assignment 1 : Decision Tress using `scikit-learn`

Scikit-learn provides a range of supervised and unsupervised learning algorithms via a consistent interface in Python. In this assigment you'll explore how to train decision trees using the `scikit-learn` library. The scikit-learn documentation can be found [here](http://scikit-learn.org/stable/documentation.html).

In this assignment we'll attempt to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains 1151 instances and 20 attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [17]:
#You may add additional import if you want
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time

In [18]:
%matplotlib inline

In [19]:
# Read the data from csv file
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

data = pd.read_csv("messidor_features.txt", names = col_names)
print(data.shape)
data.head(10)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-ed5f9f3476b3>", line 21, in <module>
    data = pd.read_csv("messidor_features.txt", names = col_names)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 678, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 440, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 787, in __init__
    self._make_engine(self.engine)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 1014, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 1708, in __init__
    self._reader = pa

FileNotFoundError: File b'messidor_features.txt' does not exist

### 1. Data preprocessing  & dimensionality reduction with PCA

Q1. Separate the feature columns from the class label column. You should end up with two separate data frames (or two lists, or two numpy arrays) - one that contains all of the feature values and one that contains the class labels.

In [20]:
# your code goes here
#print(data["label"])
x = data.drop("label",axis =1)
y = data.loc[:,"label"] 
print(x.head())
print(y.head())

   quality  prescreen  ma2  ma3  ma4  ma5  ma6  ma7   exudate8   exudate9  \
0        1          1   22   22   22   19   18   14  49.895756  17.775994   
1        1          1   24   24   22   18   16   13  57.709936  23.799994   
2        1          1   62   60   59   54   47   33  55.831441  27.993933   
3        1          1   55   53   53   50   43   31  40.467228  18.445954   
4        1          1   44   44   44   41   39   27  18.026254   8.570709   

   exudate10  exudate11  exudate12  exudate13  exudate14  exudate15    euDist  \
0   5.270920   0.771761   0.018632   0.006864   0.003923   0.003923  0.486903   
1   3.325423   0.234185   0.003903   0.003903   0.003903   0.003903  0.520908   
2  12.687485   4.852282   1.393889   0.373252   0.041817   0.007744  0.530904   
3   9.118901   3.079428   0.840261   0.272434   0.007653   0.001531  0.483284   
4   0.410381   0.000000   0.000000   0.000000   0.000000   0.000000  0.475935   

   diameter  amfm_class  
0  0.100025           1 

Q2. Use `sklearn.preprocessing.StandardScaler` to standardize the dataset’s features (mean = 0 and variance = 1). Only standardize the the features, not the class labels! This will be required for running the principal component analysis (PCA) dimensionality reduction later. Note that `StandardScaler` returns a numpy array.

In [21]:
# your code goes here
scaler = StandardScaler()
std_features = scaler.fit_transform(x)
print(std_features)

#create a new table with standardized data and labels
columns = list(x.columns.values)
df = pd.DataFrame(std_features,columns = columns)
new_data = df.join(y)
new_data.head()

[[ 0.05905386  0.2982129  -0.6414863  ... -1.29476283 -0.46865568
   1.40504812]
 [ 0.05905386  0.2982129  -0.56339113 ... -0.08216786  2.00605415
  -0.7117194 ]
 [ 0.05905386  0.2982129   0.92041699 ...  0.27428264  1.1215164
  -0.7117194 ]
 ...
 [ 0.05905386 -3.35330894  0.41279842 ...  1.33436273  1.19371332
  -0.7117194 ]
 [ 0.05905386  0.2982129   0.0223226  ... -1.32796165 -0.09707846
   1.40504812]
 [ 0.05905386  0.2982129  -1.22720003 ...  1.17603538 -1.08570243
  -0.7117194 ]]


,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class,label
0,0.059054,0.298213,-0.641486,-0.618782,-0.576463,-0.630029,-0.551116,-0.473745,-0.242917,-0.246003,-0.296966,-0.271509,-0.218324,-0.194409,-0.205124,-0.186169,-1.294763,-0.468656,1.405048,0
1,0.059054,0.298213,-0.563391,-0.535778,-0.576463,-0.677410,-0.653676,-0.539992,-0.109250,0.032972,-0.465224,-0.408593,-0.224256,-0.197212,-0.205175,-0.186281,-0.082168,2.006054,-0.711719,0
2,0.059054,0.298213,0.920417,0.958299,1.046665,1.028299,0.936006,0.784951,-0.141383,0.227196,0.344463,0.769037,0.335538,0.152330,-0.110043,-0.164808,0.274283,1.121516,-0.711719,1
3,0.059054,0.298213,0.647084,0.667784,0.783456,0.838776,0.730886,0.652456,-0.404199,-0.214977,0.035830,0.316953,0.112573,0.056919,-0.195765,-0.199541,-1.423814,0.354501,-0.711719,0
4,0.059054,0.298213,0.217561,0.294265,0.388641,0.412349,0.525766,0.387468,-0.788069,-0.672306,-0.717335,-0.468311,-0.225828,-0.200905,-0.214968,-0.208100,-1.685874,0.844102,-0.711719,1


Q3 . Split your dataset into training and test sets (80% - 20% split). Use `sklearn.model_selection.train_test_split` to help you in this task. You should be working with your standardized features from here forward. Display how many records are in the training set and how many are in the test set.

In [22]:
# your code goes here
train_features, test_features, train_label, test_label  = train_test_split(std_features,y, train_size = 0.8)
print("\n")
print("train features length is", len(train_features))
print(train_features)
print("\n")
print("test_features length is ",len(test_features))
print(train_features)



train features length is 920
[[ 0.05905386  0.2982129  -1.11005728 ...  0.14166566 -0.73988196
  -0.7117194 ]
 [ 0.05905386  0.2982129  -0.6414863  ... -0.60810708  0.80736222
   1.40504812]
 [ 0.05905386  0.2982129  -0.87577179 ...  0.79736912 -0.00514586
   1.40504812]
 ...
 [ 0.05905386  0.2982129   0.84232182 ...  0.02434651  1.34106195
  -0.7117194 ]
 [ 0.05905386  0.2982129   0.41279842 ... -0.23949675 -0.52173017
  -0.7117194 ]
 [ 0.05905386  0.2982129  -0.05577257 ... -1.60549809  0.13166592
  -0.7117194 ]]


test_features length is  231
[[ 0.05905386  0.2982129  -1.11005728 ...  0.14166566 -0.73988196
  -0.7117194 ]
 [ 0.05905386  0.2982129  -0.6414863  ... -0.60810708  0.80736222
   1.40504812]
 [ 0.05905386  0.2982129  -0.87577179 ...  0.79736912 -0.00514586
   1.40504812]
 ...
 [ 0.05905386  0.2982129   0.84232182 ...  0.02434651  1.34106195
  -0.7117194 ]
 [ 0.05905386  0.2982129   0.41279842 ... -0.23949675 -0.52173017
  -0.7117194 ]
 [ 0.05905386  0.2982129  -0.0557725

Q4. PCA is affected by the scale of the features that is why it is important to standardize the dataset first. The principle components generated by PCA are sensitive to the shape of the data in d-dimensional space. 
* Carry out a principal components analysis using `sklearn.decomposition.PCA` Note that you are fitting PCA on the **training set** only (NOT the test set).
* Use the `pca.explained_variance_ratio_` field to determine how many principal components are needed so that 95% variance is retained. 
* Reduce the PCA-transformed-dataset to this number of columns and you'll use the resulting dataset for subsequent tasks.

* Once the training set's dimensionality has been reduced with PCA, transform the **test set** to the principal component space that was created. (Do not fit a new PCA. Use the same one that was created with the training set.)

In [23]:
# your code goes here
from sklearn.decomposition import PCA
#find the number of raduction
pca = PCA(n_components = 10)
pca_features = pca.fit_transform(train_features)
pcadf = pd.DataFrame(data = pca_features)
print(pcadf.head())
#get the final database
finalDf = pd.concat([pcadf, data[['label']]], axis = 1)
print(finalDf.head())
#get a pca looking for only 95% variance for components are retained
pca = PCA(.95)
#fit only the test_features

pca.fit(test_features)
print(pca.n_components_)
print(pca.explained_variance_ratio_)
#transform test and train
test_features = pca.transform(test_features)
train_features = pca.transform(train_features)


          0         1         2         3         4         5         6  \
0 -2.063653 -1.449128  1.459714 -0.721155 -0.537560 -0.710435 -0.744503   
1 -2.856053  1.580877 -2.782294  1.026801 -0.133230  0.679909  0.177250   
2 -2.283809 -0.578248 -0.064578 -0.716522  0.064683  0.774331  0.658177   
3  3.967881 -0.714961  0.137227  0.091319 -0.699504 -0.879536  0.580610   
4  2.501016 -1.068799  0.508441 -0.516080 -0.312433 -0.320911 -0.061848   

          7         8         9  
0  0.029282  0.067854 -0.107069  
1 -0.065264 -0.209181 -1.365485  
2 -0.111982  0.281289 -0.519497  
3 -0.029598 -0.088080  0.316492  
4  0.015689  0.186533 -0.008338  
          0         1         2         3         4         5         6  \
0 -2.063653 -1.449128  1.459714 -0.721155 -0.537560 -0.710435 -0.744503   
1 -2.856053  1.580877 -2.782294  1.026801 -0.133230  0.679909  0.177250   
2 -2.283809 -0.578248 -0.064578 -0.716522  0.064683  0.774331  0.658177   
3  3.967881 -0.714961  0.137227  0.091319 -0.

In [24]:
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
# default solver is incredibly slow which is why it was changed to 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(train_features, train_label)
logisticRegr.score(test_features, test_label)


0.6493506493506493

### 2. Training Decision Trees in `scikit-learn`

Q5. Use `sklearn.tree.DecisionTreeClassifier` to fit a decision tree classifier on the training set. Use entropy as the split criterion. 

In [25]:
from sklearn.tree import DecisionTreeClassifier

# your code goes here
clf = DecisionTreeClassifier(criterion = "entropy")
clf.fit(train_features,train_label)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Q6. Now that the tree has been learned from the training data, we can run the test data through and predict classes for our test data. Use the `predict` method of `DecisionTreeClassifier` to classify the test data. Then use `sklearn.metrics.accuracy_score` to print out the accuracy of the classifier on the test set.

In [26]:
# your code goes here
from sklearn.metrics import accuracy_score

#return y shape
y_predict = clf.predict(test_features)
accuracy_score(y_true = test_label ,y_pred = y_predict)


0.6320346320346321

Q7. Note that the DecisionTree classifier has many parameters that can be set. Try tweaking parameters like split criterion, max_depth, min_impurity_decrease, min_samples_leaf, min_samples_split, etc. to see how they affect accuracy.

In [27]:
# your code goes here
# your code goes here
clf2 = DecisionTreeClassifier(criterion = "entropy", max_depth=10, min_samples_leaf = 11)
clf2.fit(train_features,train_label)
#return y shape
y_predict2 = clf2.predict(test_features)
accuracy = accuracy_score(y_true = test_label ,y_pred = y_predict2)
print(accuracy)


0.6320346320346321


### 3. Using K-fold Cross Validation

You have now built a decision tree and tested it's accuracy using the "holdout" method. But as discussed in class, this is not sufficient for estimating generalization accuracy. Instead, we should use Cross Validation to get a better estimate of accuracy. You will do that next.

Q7. Use `sklearn.model_selection.cross_val_score` to perform 10-fold cross validation on your decision tree. Report the accuracy of the model. For this step, revert back to using the data before it underwent PCA. So you want to use the standardized data that came out of Q2. 

In [31]:
# your code goes here
from sklearn.model_selection import cross_val_score

accy = print(cross_val_score(clf, std_features,y=y, cv=10))
print(accy)

[0.55172414 0.5826087  0.66956522 0.63478261 0.62608696 0.64347826
 0.60869565 0.6        0.66956522 0.5826087 ]
None


Q8. Now we want to tune our model to use the best parameters to avoid overfitting to our training data. Grid search is an approach to parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid. 
* Use `sklearn.model_selection.GridSearchCV` to find the best `max_depth`, `max_features`, and `min_samples_leaf` for your tree. Use 'accuracy' for the scoring criteria.
* Try the values [5,10,15,20] for `max_depth` and `min_samples_leaf`. Try [5,10,15] for `max_features`. 
* Use a 5-fold cross validation. 
* Print out the best value for each of the tested parameters.
* Print out the accuracy of the model with these best values.

In [29]:
# your code goes here
from sklearn.model_selection import GridSearchCV
# {'kernel':('linear', 'rbf'), 'C':[1, 10]}
param = {'max_depth':[5,10,15,20],'min_samples_leaf':[5,10,15,20],'max_features':[5,10,15]}
print(std_features)
grid = GridSearchCV(clf, param, scoring= accy ,cv=5)
grid.fit(std_features,y)
#grid.cv_results_
print(grid.best_params_)
print(grid.best_score_)

[[ 0.05905386  0.2982129  -0.6414863  ... -1.29476283 -0.46865568
   1.40504812]
 [ 0.05905386  0.2982129  -0.56339113 ... -0.08216786  2.00605415
  -0.7117194 ]
 [ 0.05905386  0.2982129   0.92041699 ...  0.27428264  1.1215164
  -0.7117194 ]
 ...
 [ 0.05905386 -3.35330894  0.41279842 ...  1.33436273  1.19371332
  -0.7117194 ]
 [ 0.05905386  0.2982129   0.0223226  ... -1.32796165 -0.09707846
   1.40504812]
 [ 0.05905386  0.2982129  -1.22720003 ...  1.17603538 -1.08570243
  -0.7117194 ]]
{'max_depth': 5, 'max_features': 10, 'min_samples_leaf': 15}
0.6455256298870548


Q9. To perform the nested cross-validation that we discussed in class, you'll now need to pass the `GridSearchCV` into a `cross_val_score`. 

What this does is: the `cross_val_score` splits the data in to train and test sets for the first fold, and it passes the train set into `GridSearchCV`. `GridSearchCV` then splits that set into train and validation sets for k number of folds (the inner CV loop). The hyper-parameters for which the average score over all inner iterations is best, is reported as the `best_params_`, `best_score_`, and `best_estimator_`(best decision tree). This best decision tree is then evaluated with the test set from the `cross_val_score` (the outer CV loop). And this whole thing is repeated for the remaining k folds of the `cross_val_score` (the outer CV loop). 

That is a lot of explanation for a very complex (but IMPORTANT) process, which can all be performed with a single line of code!

Be patient for this one to run. The nested cross-validation loop can take some time. An asterisk [\*] next to the cell indicates that it is still running.

Print the accuracy of your tuned, cross-validated model. This is the official accuracy that you would report for your model.

In [30]:
# your code goes here
print(cross_val_score(grid,std_features,y=y, cv=10))

[0.69827586 0.57391304 0.62608696 0.55652174 0.66956522 0.72173913
 0.62608696 0.5826087  0.62608696 0.60869565]


Q10. Our accuracy rate isn't very good. We wouldn't want to use this model in the real world to actually diagnose patients because it is wrong about 40% of the time! What can we do to improve the accuracy of the model? Answer as a comment:

In [36]:
'''
Improving the accuracy of the model would require more data to distinguish form. 
This could also help the missing values or filling the missing values and removing
outlier values would improve the model. 
'''

'\nyour answer goes here \n'